Mounted at /content/drive


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import random

import torch.nn as nn
import torch.optim as optim
import torch.distributions as dist
import pickle
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import pandas as pd

%matplotlib inline

In [4]:
file_loaded=open('/content/drive/MyDrive/Colab Notebooks/LSTM/result_10_4_4_4.pickle','rb')
loaded_data=pickle.load(file_loaded)

loaded_data_tensor=torch.tensor(loaded_data,dtype=torch.float32)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/LSTM/result_10_4_4_4.pickle'

In [ ]:
loaded_data[0]

[array([10.        ,  4.        ,  4.        ,  4.        ,  2.        ,
         1.        ,  3.        ,  2.        ,  0.2       ,  0.25      ,
         0.75      ,  0.5       ,  1.50588235,  1.88235294,  5.64705882,
         3.76470588,  0.        ,  1.        ,  3.        ,  2.        ]),
 array([10.  ,  4.  ,  4.  ,  4.  ,  0.  ,  1.  ,  3.  ,  2.  ,  0.  ,
         0.25,  0.75,  0.5 ,  0.  ,  1.6 ,  4.8 ,  3.2 ,  3.  ,  3.  ,
         3.  ,  2.  ]),
 array([10.        ,  4.        ,  4.        ,  4.        ,  3.        ,
         3.        ,  3.        ,  2.        ,  0.3       ,  0.75      ,
         0.75      ,  0.5       ,  2.29565217,  5.73913043,  5.73913043,
         3.82608696,  2.        ,  4.        ,  4.        ,  3.        ]),
 array([10.        ,  4.        ,  4.        ,  4.        ,  2.        ,
         4.        ,  4.        ,  3.        ,  0.2       ,  1.        ,
         1.        ,  0.75      ,  1.41016949,  7.05084746,  7.05084746,
         5.28813559,  0.   

In [ ]:
train_input=loaded_data_tensor[:900,:,:16]
test_input=loaded_data_tensor[900:,:,:16]

train_target=loaded_data_tensor[:900,:,16:].long()
test_target=loaded_data_tensor[900:,:,16:].long()

In [ ]:
class LSTM(nn.Module):
    def __init__(self, in_dim, hid_dim, out_dim):
        super(LSTM, self).__init__()
        # 入力層: 入力サイズ in_dim から出力サイズ 64 への線形変換
        self.linear1 = nn.Linear(in_dim, 64)
        # LSTM 層: 入力サイズ 64、隠れサイズ 16
        self.lstm = nn.LSTM(input_size=64, hidden_size=16, batch_first=True)
        # 出力層: 隠れサイズ 16 から最終出力サイズ out_dim (11) への線形変換
        self.linear2 = nn.Linear(16, out_dim)
        # 活性化関数
        self.tanh = nn.Tanh()
        # self.h0=torch.zeros(1,1,16)
        # self.c0=torch.zeros(1,1,16)

    def forward(self, x):
        # 最初の線形層 + Tanh
        x = self.tanh(self.linear1(x))  # 出力サイズ: [バッチサイズ, シーケンス長, 64]
        # LSTM 層
        lstm_out, (self.h0,self.c0) = self.lstm(x,(self.h0,self.c0))      # 出力サイズ: [バッチサイズ, シーケンス長, 16]
        # LSTM の最終時刻の出力を取得
        # final_output = lstm_out[:, -1, :]  # 最終時刻の出力のみを使用, サイズ: [バッチサイズ, 16]
        # 最終線形層
        x = self.linear2(lstm_out)   # 出力サイズ: [バッチサイズ, out_dim]
        return x

    def reset(self):
        self.h0=torch.zeros(1,16)
        self.c0=torch.zeros(1,16)


In [ ]:
# ハイパーパラメタの決定
in_dim=16
hid_dim=16
out_dim=11
learning_rate = 0.001
num_epochs = 100

# lstmモデル、損失関数、最適化関数の初期化
model_list, optimizer_list = [], []
criterion = nn.CrossEntropyLoss()
for _ in range(4):
    model = LSTM(in_dim, hid_dim, out_dim)
    model_list.append(model)
    optimizer_list.append(optim.Adam(model.parameters(), lr=learning_rate))

In [ ]:
# 学習ループ
for player in range(2):
  for epoch in range(num_epochs):
      model_list[player].train()
      epoch_loss = 0.0  # 各エポックの損失を累積する変数

      for i in range(train_input.size(0)):
          inputs = train_input[i]    # サイズ: [9, 16]
          targets = train_target[i,:,player].long() # サイズ: [9]
          model_list[player].reset()
          # フォワードプロパゲーション
          outputs = model_list[player](inputs)  # 出力サイズ: [9, 11]
          # 損失計算
          loss = criterion(outputs, targets)
          # バックプロパゲーションと最適化
          optimizer_list[player].zero_grad()
          loss.backward()
          optimizer_list[player].step()

          epoch_loss += loss.item()
      if(epoch%10==0):
          print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss / train_input.size(0):.4f}")

Epoch [1/100], Loss: 1.6195
Epoch [11/100], Loss: 1.2357
Epoch [21/100], Loss: 1.2151
Epoch [31/100], Loss: 1.2028
Epoch [41/100], Loss: 1.1951
Epoch [51/100], Loss: 1.1872
Epoch [61/100], Loss: 1.1824
Epoch [71/100], Loss: 1.1790
Epoch [81/100], Loss: 1.1738
Epoch [91/100], Loss: 1.1708
Epoch [1/100], Loss: 0.3439
Epoch [11/100], Loss: 0.1321
Epoch [21/100], Loss: 0.1304
Epoch [31/100], Loss: 0.1271
Epoch [41/100], Loss: 0.1256
Epoch [51/100], Loss: 0.1255
Epoch [61/100], Loss: 0.1237
Epoch [71/100], Loss: 0.1221
Epoch [81/100], Loss: 0.1205
Epoch [91/100], Loss: 0.1195


In [ ]:
print(train_input[0][:1])
print(model_list[0](train_input[0][:1:]))

tensor([[10.0000,  4.0000,  4.0000,  4.0000,  2.0000,  1.0000,  3.0000,  2.0000,
          0.2000,  0.2500,  0.7500,  0.5000,  1.5059,  1.8824,  5.6471,  3.7647]])
tensor([[  5.6307,   5.2688,   5.0981,   0.4679,  -1.9048,  -3.6072,  -3.9997,
          -6.7206, -10.6703, -12.9718,  -9.2003]], grad_fn=<AddmmBackward0>)


In [ ]:
class GN(nn.Module):
  def __init__(self,in_dim,out_dim):
    super().__init__()
    self.phi_e = nn.Linear(32+in_dim*2+32,32)
    self.phi_v = nn.Linear(32+in_dim+32,32)
    self.phi_u = nn.Linear(32*3,32)

    self.phi_e_p = nn.Linear(32*4,32)
    self.phi_v_p = nn.Linear(32*3,out_dim)

    self.tanh = nn.Tanh()

  def forward(self,x):
    e,v,u = torch.zeros(4,4,32),torch.zeros(4,32),torch.zeros(32)
    output = torch.zeros(4)
    for i in range(4):
      for j in range(4):
          e[i][j] = self.tanh(self.phi_e(torch.cat((torch.zeros(32),x[game][i],x[game][j],u),dim=0)))

    for i in range(4):
      v[i] = self.tanh(self.phi_v(torch.cat((torch.sum(e,dim=1)[i],x[i],u),dim=0)))

    u = self.tanh(self.phi_u(torch.cat((torch.sum(torch.sum(e,dim=2),dim=1),torch.sum(v,dim=1),u))))

    e_p = torch.zeros(4,4,32)

    for i in range(4):
      for j in range(4):
        e_p[i][j] = self.tanh(self.phi_e_p(torch.cat((e[i][j],v[i],v[j],u),dim=0)))

    for i in range(4):
      output[i] = self.tanh(self.phi_v_p(torch.cat((torch.sum(e_p[i],dim=1),v[i],u))))

    return output



In [ ]:
# ハイパーパラメタの決定
gn_in_dim=3
gn_out_dim=1
learning_rate = 0.001
gn_epochs = 100
sample_num = 1000

gn_model = GN(gn_in_dim,gn_out_dim)
gn_optimizer = optim.SGD(gn_model.parameters(), lr=learning_rate)

def Prob(x):
  x = x[0]
  prob = [0]*(len(x))
  for i in range(len(prob)):
    prob[i] = prob[i-1]+torch.exp(x[i])

  s = random.uniform(0,prob[-1])
  for i in range(len(prob)):
    if s <=  prob[i]:
      return i


def criterion():
  return



In [ ]:
print(Prob(model_list[0](train_input[0][:1:])))

x = torch.zeros(1,12)
x[0][:4] = torch.tensor([10,4,4,4])
print(x)

2
tensor([[10.,  4.,  4.,  4.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])


In [ ]:
for epoch in range(gn_epochs):
  epoch_loss = 0.0
  gn_model.train()
  loss = torch.tensor([])
  for _ in range(sample_num):
      x = torch.zeros(1,12)
      x[0][:4] = torch.tensor([10,4,4,4])
      x.tensor.requires_grad = True
      log = torch.zeros(10,12)
      log[0] = x
      for player in range(4):
        model_list[player].reset()
      for i in range(10):
        for player in range(4):








In [ ]:
print(train_target[0,:,].long())

tensor([[0, 1, 3, 2],
        [3, 3, 3, 2],
        [2, 4, 4, 3],
        [0, 4, 4, 3],
        [4, 4, 4, 4],
        [3, 4, 4, 4],
        [3, 4, 4, 4],
        [3, 4, 4, 4],
        [0, 4, 4, 4]])


In [ ]:
x = torch.zeros(5,5,5)
for i in range(5):
  for j in range(5):
    for k in range(5):
      x[i][j][k] = i*25+5*j+k

print(x)
print(torch.sum(torch.sum(x,dim=2),dim=1))

tensor([[[  0.,   1.,   2.,   3.,   4.],
         [  5.,   6.,   7.,   8.,   9.],
         [ 10.,  11.,  12.,  13.,  14.],
         [ 15.,  16.,  17.,  18.,  19.],
         [ 20.,  21.,  22.,  23.,  24.]],

        [[ 25.,  26.,  27.,  28.,  29.],
         [ 30.,  31.,  32.,  33.,  34.],
         [ 35.,  36.,  37.,  38.,  39.],
         [ 40.,  41.,  42.,  43.,  44.],
         [ 45.,  46.,  47.,  48.,  49.]],

        [[ 50.,  51.,  52.,  53.,  54.],
         [ 55.,  56.,  57.,  58.,  59.],
         [ 60.,  61.,  62.,  63.,  64.],
         [ 65.,  66.,  67.,  68.,  69.],
         [ 70.,  71.,  72.,  73.,  74.]],

        [[ 75.,  76.,  77.,  78.,  79.],
         [ 80.,  81.,  82.,  83.,  84.],
         [ 85.,  86.,  87.,  88.,  89.],
         [ 90.,  91.,  92.,  93.,  94.],
         [ 95.,  96.,  97.,  98.,  99.]],

        [[100., 101., 102., 103., 104.],
         [105., 106., 107., 108., 109.],
         [110., 111., 112., 113., 114.],
         [115., 116., 117., 118., 119.],
        

In [ ]:
in_dim_M = 12
out_dim_M = 4


In [ ]:
def SGD(epochs, learning_rate, theta_init, batch_size):
    theta = torch.Tensor(theta_init)
    scores = []
    grads = []
    thetas = [theta_init]
    theta_after = theta
    for epoch in range(mecha_epochs):
        grad,score = np.zeros(batch_size),np.zeros(batch_size)
        for batch in range(batch_size):
            for _ in range(10):


        Grad = np.mean(grad)
        Score = np.mean(score)
        theta_after = theta - mecha_learning_rate * Grad
        scores.append(Score)
        grads.append(Grad)
        thetas.append(theta.detach())
        if epoch % 10 == 0:
              print('epoch = {:3d}, theta = {:6f}, loss = {:6f}, gradient = {:6f}\r'.format(epoch, float(theta), float(Score), float(Grad)), end='')
    return scores, grads, thetas